# Iteración de Valores

En este ejercicio vamos a implementar el primer método para solucionar Procesos de Decisión de Markov (MDPs). El método a implementar es la iteración de valores.

La iteración de valores esta basada en la fórmula:

![value_iteration](https://raw.githubusercontent.com/FLAGlab/isis4222-rl/a502e264157729fcb8cc00d484e4a8e8e4734a15/week4/img/value_iteration.png)

Para resolver los MDPs crearemos un archivo `value_iteration.py` el cual utilizaremos para solucionar el ambiente de Gridworld.

**Task 1**
1. Implemente la classe `ValueIteration` con cuatro atributos:
    - `mdp` que corresponde al MDP a resolver (e.g., Gridworld) 
    - `discount` que corresponde al factor de decuento a utilizar, `0.9` por defecto.
    - `iterations` que corresponde a el número de iteraciones a realizar.
    - `values` que corresponde a los valores calculados para los estados del MDP.

2. El comportamiento del agente (de iteración de valores) esta dado por los métodos:
    - `run_value_iteration` que no recibe ningún parámatetro y ejecuta el algoritmo de iteración de valores para la solución del MDP.
    - `get_value` recibe un estado y retorna el valor correspondiende para dicho estado.
    - `compute_qvalue_from_values` recibe un estado y una acción y calcula el q valor correspondiente.
    - `compute_action_from_values` que calcula la acción a tomar (como la acción con el mejor valor en `values`) para un estado dado
    - `get_action` retorna la acción a tomar dado un estado (directamente como la acción de la política, sin exploración)
    - `get_qvalue` retorna el q valor dado un estado y una acción
    - `get_policy` que retorna la acción a tomar para un estado (como `get_action`). Si el estado no tiene una acción asociada a él, retorne `None`

# Entrega

Para esta tarea debe entregar: 
- La implementación de la iteración de valores para solucionar MDPs (`value_iteration.py`).
- Un documento de análisis respondiendo a las siguientes preguntas (con screenshots de la solución y las explicaciones correspondientes del comportamiento observado).
  -	Ejecute su implementación de iteración de valores para 5, 10, 15, 20, 30, 50 iteraciones, sobre el ambiente de gridworld. ¿Cuando convergen los valores o las acciones?
  -	Ejecute su implementación sobre el ambiente del puente (i.e., Bridge) durante 10 iteraciones. Qué resultado observa si kodifica el valor de descuento de 0.9 a 0.1. Explique si los resultados cambian y porqué. 
  
  Recuerde que el ambiente del puente se define con la matriz de `3x7` donde las filas 1 y 3 tienen recompensa -100 entre las columnas 2 y 6. La fila 2 corresponde a el puente, con entrada en la casilla `(2,1)` y salida en la casilla `(2,7)` con recompensa 100, como se muestra en la figura

  ![bridge](https://raw.githubusercontent.com/FLAGlab/isis4222-rl/a502e264157729fcb8cc00d484e4a8e8e4734a15/week4/img/bridge.png)


In [1]:


from assignment_agent_iteration.grid_world import EnvironmentWorld
from assignment_agent_iteration.value_iteration import ValueIteration
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

grid_world = EnvironmentWorld([
    ['S'] + [' '] * 9,
    [' '] * 10,
    [' ', '#', '#', '#', '#', ' ', '#', '#', '#', ' '],
    [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', '#', '-1', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', '#', '+1', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', '#', '-1', '-1', ' ', ' ', ' '],
    [' '] * 10,
    [' '] * 10
])

bridge_world = EnvironmentWorld([
    ['#', '-100', '-100', '-100', '-100', '-100', '#'],
    ['+1', 'S', ' ', ' ', ' ', ' ', '+100'],
    ['#', '-100', '-100', '-100', '-100', '-100', '#'],
])

print(grid_world)
print('\n\n')
print(bridge_world)

    0  1  2  3  4   5   6  7  8  9
0  SC                             
1                                 
2      #  #  #  #       #  #  #   
3               #                 
4               #  -1             
5               #  +1             
6               #                 
7               #  -1  -1         
8                                 
9                                 



    0     1     2     3     4     5     6
0   #  -100  -100  -100  -100  -100     #
1  +1    SC                          +100
2   #  -100  -100  -100  -100  -100     #


In [2]:
def print_value_iteration(value_iteration, previous_values=None, previous_iterations=None):
    print(f'\n{"_" * 8} iterations={value_iteration.iterations} discount={value_iteration.discount} {"_" * 8}')
    current_value = pd.DataFrame(value_iteration.values)
    if previous_values is not None:
        average_square_error = ((current_value - previous_values) ** 2).fillna(0).values.sum() / (
                current_value.count().sum() * (value_iteration.iterations - previous_iterations))
        print(f'average_square_error (measure of convergence) = {average_square_error}')
    print("___values___")
    print(value_iteration)
    print("___policy___")
    print(value_iteration.get_full_policy().map(lambda action: action.name if action else 'NONE'))


In [3]:
iteration_numbers = [5, 10, 15, 20, 30, 50]
previous_values = None
previous_iterations = None
for iteration_number in iteration_numbers:
    value_iteration = ValueIteration(grid_world, iterations=iteration_number, discount=0.9)
    value_iteration.run_value_iteration()
    print_value_iteration(value_iteration, previous_values, previous_iterations)
    previous_values = pd.DataFrame(value_iteration.values)
    previous_iterations = iteration_number


________ iterations=5 discount=0.9 ________
___values___
     0    1    2    3    4         5         6         7         8         9
0  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.000000
1  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.000000
2  0.0  NaN  NaN  NaN  NaN  0.073732       NaN       NaN       NaN  0.000000
3  0.0  0.0  0.0  0.0  NaN  0.270407  0.539200  0.403257  0.243258  0.000000
4  0.0  0.0  0.0  0.0  NaN  1.714597  1.089359  0.822692  0.380470  0.177867
5  0.0  0.0  0.0  0.0  NaN  2.064785  1.985762  1.156169  0.652458  0.223725
6  0.0  0.0  0.0  0.0  NaN  2.138855  1.587304  1.094122  0.576696  0.290805
7  0.0  0.0  0.0  0.0  NaN  0.791830  0.795399  0.464834  0.315038  0.000000
8  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.193870  0.000000  0.000000
9  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.000000  0.000000  0.000000
___policy___
    0     1     2     3     4      5      6     7     8     9
0  UP    UP    UP   

## Convergencia de política y valores

En este caso podemos observar que entre las iteraciones 30 y 50 la política converge a su estado final; Sin embargo, al observar los cambios en los valores podemos ver que estos fueron lentos en las últimas iteraciones

In [4]:
value_iteration_09 = ValueIteration(bridge_world, iterations=10, discount=0.9)
value_iteration_09.run_value_iteration()
print_value_iteration(value_iteration_09)

value_iteration_01 = ValueIteration(bridge_world, iterations=10, discount=0.1)
value_iteration_01.run_value_iteration()
print_value_iteration(value_iteration_01)



________ iterations=10 discount=0.9 ________
___values___
          0           1           2           3           4           5           6
0       NaN -115.596189 -100.334199    1.791989  126.260944  255.750562         NaN
1  6.861894  -62.756976   -8.971765  118.124531  283.356827  497.878666  686.189404
2       NaN -107.159597  -61.179133   49.405868  182.983836  309.567928         NaN
___policy___
      0     1      2      3      4      5     6
0  NONE  DOWN  RIGHT  RIGHT  RIGHT   DOWN  NONE
1  DOWN  LEFT  RIGHT  RIGHT  RIGHT  RIGHT    UP
2  NONE    UP     UP     UP     UP     UP  NONE

________ iterations=10 discount=0.1 ________
___values___
          0          1          2          3          4          5           6
0       NaN -43.601096 -43.755878 -43.755017 -43.514521 -39.550402         NaN
1  1.111111 -30.950875 -33.504777 -33.413169 -29.445417  35.842294  111.111111
2       NaN -43.605918 -43.760200 -43.743728 -43.383354 -39.426523         NaN
___policy___
      0     

## Efecto de descuento

Al reducir el descuento el agente converge de forma más rápida, ya que los valores de `descuento ** t` aproximan `0` de forma más rápida. Adicionalmente, el agente opta por estrategias de más corto plazo, en este caso cuando está del lado izquierdo ignora el 100 del otro lado del puente.